# Particle flow network

In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import matplotlib.pyplot as plt
# import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.layers import Conv1D, Flatten, Dense, Input, concatenate, Masking, LSTM, TimeDistributed, Bidirectional, RepeatVector
# from tensorflow.keras import regularizers 
# from tensorflow.keras import initializers

import energyflow as ef
from energyflow.archs import PFN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, remap_pids, to_categorical
import h5py 
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="1"


%matplotlib inline 

dataset = h5py.File('./../../../../../data/hongtao/variables_tt_re2.h5','r')

X_keys = ['j1_pt', 'j1_eta', 'j1_phi', 'j1_m', 'j1_DL1r', 'j2_pt', 'j2_eta', 'j2_phi', 'j2_m', 'j2_DL1r', 'j3_pt', 'j3_eta', 'j3_phi', 'j3_m', 'j3_DL1r', 'j4_pt', 'j4_eta', 'j4_phi', 'j4_m', 'j4_DL1r', 'j5_pt', 'j5_eta', 'j5_phi', 'j5_m', 'j5_DL1r', 'j6_pt', 'j6_eta', 'j6_phi', 'j6_m', 'j6_DL1r', 'lep_pt', 'lep_eta', 'lep_phi', 'met_met', 'met_phi']
Y_keys = ['th_pt', 'th_eta','th_phi', 'tl_pt', 'tl_eta', 'tl_phi', 'wl_pt', 'wl_eta', 'wl_phi']
phi_keys = list(filter(lambda a: 'phi' in a, dataset.keys()))
eta_keys = list(filter(lambda a: 'eta' in a, dataset.keys()))
pt_keys =  list(filter(lambda a: 'pt' in a, dataset.keys()))
m_keys = list(filter(lambda a: 'm' in a, dataset.keys()))
DL1r_keys = list(filter(lambda a: 'DL1r' in a, dataset.keys()))


Y_length = len(Y_keys)
X_length = len(X_keys)
crop0 =  200000 

In [2]:
import scale_variables
import shape_timesteps

# How to scale each input/output
def get_methods(keys):
    types = {'pt':'cartbox', 'eta':'meanmax', 'phi':'phi_pi', 'DL1r':'meanmax', 'm':'meanmax', 'met':'meanmax'}
    var_types = [key.split('_')[1] for key in keys]
    methods = [types[var] for var in var_types]
    return methods

X_methods = get_methods(X_keys)
Y_methods = get_methods(Y_keys)

Scaler = scale_variables.Scale_variables()
(X_total, X_maxmean), X_names = Scaler.scale_arrays(X_keys, X_methods, True)
(Y_total, Y_maxmean), Y_names = Scaler.scale_arrays(Y_keys, Y_methods, True)

error = Scaler.test_inverse(Y_keys, Y_methods, True)
print('Max scaling error: {}'.format(error))

Max scaling error: 3.410605131648481e-13


In [3]:
# Create X and test array
split = int(np.floor(0.8*crop0)) # 80/20 split 

trainY, testY = Y_total[0:split,:], Y_total[split:,:]

timestep_builder = shape_timesteps.Shape_timesteps()
totalX_jets, totalX_other = timestep_builder.reshape_X(X_total, X_names, True,True)

mask = timestep_builder.create_mask()
totalX_jets = mask*totalX_jets - 2*(1-mask)
trainX_jets, testX_jets = totalX_jets[0:split,:,:], totalX_jets[split:,:,:]
trainX_other, testX_other = totalX_other[0:split,:], totalX_other[split:,:]

trainX_other = np.reshape(trainX_other, (trainX_other.shape[0], 1, -1))
testX_other = np.reshape(testX_other, (testX_other.shape[0], 1, -1))
trainX = np.concatenate([trainX_other, trainX_jets], axis=1)
testX = np.concatenate([testX_other, testX_jets], axis=1)
# Save some memory 
del totalX_jets
del totalX_other 

print(trainX.shape, trainY.shape)
print(testX.shape, testY.shape)

(160000, 7, 5) (160000, 9)
(40000, 7, 5) (40000, 9)


In [4]:
# Histograms of total  Y variables 
show = False

other_bins = np.linspace(-1, 1, 40)
phi_bins = np.linspace(-0.1, 0.1, 40)
pt_bins = np.linspace(-0.3, 1, 40)
Y_bins = [phi_bins if 'phi' in name else pt_bins if 'pt' in name else other_bins for name in Y_names]

if show:
    plt.figure(figsize=(6,6*Y_total.shape[1]))
    for i in range(0, Y_total.shape[1]):
        plt.subplot(Y_total.shape[1], 1, i+1)
        bins = Y_bins[i]
        plt.hist(Y_total[:,i], bins, histtype='step')
        plt.xlabel(Y_names[i])
        plt.ylabel('Frequency')

In [5]:
# Histograms of X variables (without the mask)
show = False

if show:
    plt.figure(figsize=(6,6*X_total.shape[1]))
    for i in range(0, X_total.shape[1]):
        plt.subplot(X_total.shape[1], 1, i+1)
        plt.hist(X_total[:,i], 40, histtype='step')
        plt.xlabel(X_names[i])
        plt.ylabel('Frequency')

 

In [6]:
# network architecture parameters
Phi_sizes, F_sizes = (100, 100, 256), (256, 128)
# Phi_sizes, F_sizes = (100, 100, 256), (100, 100, 100)
optimizer = keras.optimizers.Adam(learning_rate=5e-5)

pfn = PFN(input_dim=trainX.shape[-1], Phi_sizes=Phi_sizes, F_sizes=F_sizes, mask_val=-2, output_dim=trainY.shape[-1], output_act='linear', loss='mse', optimizer=optimizer, metrics=['mse'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, None, 5)]    0                                            
__________________________________________________________________________________________________
tdist_0 (TimeDistributed)       (None, None, 100)    600         input[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, None, 100)    0           tdist_0[0][0]                    
__________________________________________________________________________________________________
tdist_1 (TimeDistributed)       (None, None, 100)    10100       activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
# Fit model

# network training parameters
num_epoch = 256
batch_size = 500
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = pfn.fit(trainX, trainY,
          epochs=num_epoch,
          batch_size=batch_size, 
                  callbacks=[callback],
          validation_data=(testX, testY),
          verbose=1)

Train on 160000 samples, validate on 40000 samples
Epoch 1/256
160000/160000 [==============================] - 3s 17us/sample - loss: 0.1442 - mse: 0.1442 - val_loss: 0.0335 - val_mse: 0.0335
Epoch 2/256
160000/160000 [==============================] - 2s 11us/sample - loss: 0.0271 - mse: 0.0271 - val_loss: 0.0230 - val_mse: 0.0230
Epoch 3/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0207 - mse: 0.0207 - val_loss: 0.0192 - val_mse: 0.0192
Epoch 4/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0181 - mse: 0.0181 - val_loss: 0.0173 - val_mse: 0.0173
Epoch 5/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0166 - mse: 0.0166 - val_loss: 0.0161 - val_mse: 0.0161
Epoch 6/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0156 - mse: 0.0156 - val_loss: 0.0152 - val_mse: 0.0152
Epoch 7/256
160000/160000 [==============================] - 2s 11us/sample - loss: 0.0149 - mse:

Epoch 58/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0089 - mse: 0.0089 - val_loss: 0.0090 - val_mse: 0.0090
Epoch 59/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0090 - val_mse: 0.0090
Epoch 60/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0089 - val_mse: 0.0089
Epoch 61/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0089 - val_mse: 0.0089
Epoch 62/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0090 - val_mse: 0.0090
Epoch 63/256
160000/160000 [==============================] - 2s 11us/sample - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0089 - val_mse: 0.0089
Epoch 64/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0087 - mse: 0.0087 - val_loss: 0.0088 - val_mse: 0.0088

160000/160000 [==============================] - 2s 11us/sample - loss: 0.0078 - mse: 0.0078 - val_loss: 0.0080 - val_mse: 0.0080
Epoch 116/256
160000/160000 [==============================] - 2s 11us/sample - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 117/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 118/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0080 - val_mse: 0.0080
Epoch 119/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 120/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 121/256
160000/160000 [==============================] - 2s 11us/sample - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0079 - val_mse: 0.0079
Epoch 

Epoch 172/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0075 - val_mse: 0.0075
Epoch 173/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0075 - val_mse: 0.0075
Epoch 174/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0075 - val_mse: 0.0075
Epoch 175/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0075 - val_mse: 0.0075
Epoch 176/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0073 - mse: 0.0073 - val_loss: 0.0075 - val_mse: 0.0075
Epoch 177/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0075 - val_mse: 0.0075
Epoch 178/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0075 - val_mse:

Epoch 229/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 230/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 231/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 232/256
160000/160000 [==============================] - 2s 11us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 233/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 234/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0072 - val_mse: 0.0072
Epoch 235/256
160000/160000 [==============================] - 2s 10us/sample - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0073 - val_mse:

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(history.history['loss'], label='training')
plt.plot(history.history['val_loss'], label='validation')
plt.xlabel('epoch')
plt.ylabel('mse loss')
plt.legend()
plt.title('MSE loss')

In [ ]:
class Analysis:
    def __init__(self):
        self.crop = 100
        self.fsize = 6
        self.histogram_bins = 40
        
    def wrap_phi(self, var):
        var = var%(2*np.pi)
        var = var - 2*np.pi*(var > np.pi)
        return var
        
    def predictions_vs_sample(self, compare, true, names, wrap_phi):
        plt.figure(figsize=(self.fsize*2, self.fsize*len(names)))
        for i in range(0, len(names)):
            compare_small = compare[:self.crop,i]
            true_small = true[:self.crop,i]
            if wrap_phi and "phi" in names[i]:
                compare_small = self.wrap_phi(compare_small)
                true_small = self.wrap_phi(true_small)
            plt.subplot(len(names), 1, i+1)
            plt.plot(range(0,self.crop), compare_small, 'bo', markersize=3, label = 'Predictions')
            plt.plot(range(0,self.crop), true_small, 'ro', markersize=3, label = 'True Value')
            ym, yM = plt.ylim()
            for x in range(self.crop):
                plt.vlines(x, color='g', linestyle='-', alpha=0.2, ymin= 
                            min(compare_small[x], true_small[x]), 
                            ymax= max(compare_small[x], true_small[x]))
            plt.hlines(np.mean(true[:,i]), xmin=-20, xmax=self.crop+20, alpha=0.5)
            MSE = 1/compare[:,i].size*np.sum((compare[:,i]- true[:,i])**2)
            plt.xlabel('Sample')
            plt.xlim(0, self.crop)
            plt.ylabel(names[i])
            plt.title(names[i] + " MSE: " + str(MSE))
            plt.legend()
    
    def display_errors(self, compare, true, names, wrap_phi):
        MSE = 1/compare.size*np.sum((compare- true)**2)
        print("total MSE: " + str(MSE))
        print(" ")
        for i in range(len(names)):
            diff = compare[:,i] -true[:,i]
            if wrap_phi and "phi" in names[i]:
                diff = self.wrap_phi(diff)
            MSE = 1/compare[:,i].size*np.sum((diff)**2)
            print("{0} MSE : ".format(names[i]), '%.10f'%MSE)
    
    def difference_histogram(self, compare, true, names, wrap_phi, bins):
        plt.figure(figsize=(self.fsize*2,self.fsize*len(names)))
        for i in range(len(names)):
            plt.subplot(len(names), 1, i+1)
            diff = true[:,i] - compare[:,i]
            hist0, bin_edges = np.histogram(true[:, i], bins=40)
            if bins[i] is None:
                hbins = bin_edges
            else:
                hbins = bins[i]
            plt.hist(diff, hbins, histtype='step', color='purple', label='true - predicted', density=True)
            plt.xlabel("Difference (Mean: {0}, Std: {1})".format(np.mean(diff), np.std(diff)))
            plt.title(names[i])
            plt.legend()
            plt.ylabel('Frequency')
            
    def variable_histogram(self, compare, true, names, wrap_phi, bins): 
        plt.figure(figsize=(self.fsize*2,self.fsize*len(names)))
        for i in range(len(names)):
            plt.subplot(len(names), 1, i+1)
            compare_small = compare[:, i]
            true_small = true[:, i]
            if wrap_phi and "phi" in names[i]:
                compare_small = self.wrap_phi(compare_small)
                true_small = self.wrap_phi(true_small)
            hist0, bin_edges = np.histogram(true[:, i], bins=40)
            
            if bins[i] is None:
                hbins = bin_edges
            else:
                hbins = bins[i]
                
            plt.hist(true_small, hbins, histtype='step', color='b', label='true values', density=False)
            plt.hist(compare_small, hbins, histtype='step', color='r', label='predictions', density=False)
            plt.xlabel(names[i])
            plt.title(names[i])
            plt.legend()
            plt.ylabel('Frequency')
    
    def difference_vs_variable(self, compare, true, names, wrap_phi):
        plt.figure(figsize=(self.fsize*2,self.fsize*len(names)))
        for i in range(len(names)):
            plt.subplot(len(names), 1, i+1)
            plt.plot(true[:, i], true[:, i]-compare[:, i], 'o', color='purple', label='True - Predicted', markersize=2)
            plt.xlabel('True ' + names[i])
            plt.legend()
            plt.ylabel('Difference')
    
    def predicted_vs_true(self, compare, true, names, wrap_phi):
        plt.figure(figsize=(self.fsize*2,self.fsize*len(names)))
        for i in range(len(names)):
            plt.subplot(len(names), 1, i+1)
            plt.plot(true[:, i], compare[:, i], 'o', color='g', markersize=2)
            line = np.linspace(np.min(true[:, i]), np.max(true[:, i]), 100)
            plt.plot(line, line, color='b')
            plt.xlabel('True')
            plt.title(names[i])
            plt.ylabel('Predicted')

In [ ]:
display = Analysis()

predictions_unscaled = pfn.predict(testX)
true_unscaled = testY 

total_predictions = pfn.predict(np.append(trainX,testX,axis=0))
(Y_total, TO_maxmean0), _ = Scaler.scale_arrays(Y_keys, Y_methods, True)

predictions_origscale = Scaler.invscale_arrays(Y_keys, total_predictions, _, Y_methods, TO_maxmean0)[split:,:]
true_origscale = Scaler.invscale_arrays(Y_keys, Y_total, _, Y_methods, TO_maxmean0)[split:,:]

del Y_total
del TO_maxmean0
del _
del total_predictions

# Training scale plots 

In [ ]:
display.display_errors(predictions_unscaled, true_unscaled, Y_names, False)

In [ ]:
display.predictions_vs_sample(predictions_unscaled, true_unscaled, Y_names, False)

In [ ]:
display.variable_histogram(predictions_unscaled, true_unscaled, Y_names, False, Y_bins)

In [ ]:
display.difference_histogram(predictions_unscaled, true_unscaled, Y_names, False, Y_bins)

In [ ]:
display.predicted_vs_true(predictions_unscaled, true_unscaled, Y_names, False)

 # Original scale plots

In [ ]:
display.display_errors(predictions_origscale, true_origscale, Y_keys, True)

In [ ]:
display.predictions_vs_sample(predictions_origscale, true_origscale, Y_keys, True)

In [ ]:
display.variable_histogram(predictions_origscale, true_origscale, Y_keys, True, [None for name in Y_names])

In [ ]:
display.predicted_vs_true(predictions_origscale, true_origscale, Y_keys, True)

In [ ]:
# weights, biases = model.layers[0].get_weights()


print(model.layers)

In [ ]:
weights = model.layers[3].get_weights()
print(weights)